# Install Required Software with Conda

*Author: Alex Lewandowski; Alaska Satellite Facility*

Installs the conda environment needed to run the notebooks in this Jupyter Book.

- `hydrosar`

---
## 0. Install any missing Jupyter packages

In [ ]:
import importlib.util

packages = ['notebook', 'ipywidgets', 'nb_conda_kernels']
pkg_str = ''
for pkg in packages:
    if not importlib.util.find_spec(pkg):
        pkg_str = f'{pkg_str}{pkg} '
if pkg_str:
    !mamba install -c {pkg_str}--yes
    
    print('\n\nIf the `notebook package` was installed, you may need to restart Jupyter Lab \n'
    'before widgets in the notebook will display correctly.')  

    print('\n\nIf the `nb_conda_kernels` package was installed, you may need to restart Jupyter Lab before the \n'
         '`osl_opera_cslc_jbook_env` environment created with this notebook will be available to use \n'
          'with the notebooks in this Jupyter Book that require it')    

**Where are you running this notebook and installing these environments?**

We ask this to properly set the `--prefix` path when creating environments in OpenSARLab.


If you are working in [OpenSARLab](https://asf.alaska.edu/asf-services-open-science-lab/), we will use `~/.local/envs`, otherwise we will use your system's conda `root_prefix` path.

In [ ]:
import ipywidgets as widgets

osl_option = widgets.RadioButtons(
    options=['In OpenSARLab', 'Not in OpenSARLab'],
    description='Where are you working?',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(osl_option)

In [ ]:
from pathlib import Path

env_config_dir = Path.cwd().parent / 'jupyter_book_environment'

in_osl = 'In' in osl_option.value

if in_osl:
    conda_prefix = Path.home() / '.local/envs'
else:
    conda_info = subprocess.run(
        ['conda', 'run', '-n', 'base', 'conda', 'info', '--json'], 
        capture_output=True, 
        text=True
    )
    info = json.loads(conda_info.stdout)
    conda_prefix = Path(f"{info['root_prefix']}/envs")
print(f'conda prefix: {conda_prefix}')

---
## 1. Build the `hydrosar` Conda Environment

**Select whether to create the environment from configs with explicitly pinned package versions or from unpinned configs**

After running the following code cell:
- Select `Pinned` for maximum (but not guaranteed) stability when creating the environment on a Linux x86 system
- Select `Unpinned` to attempt installing the latest package versions or if on a non-Linux x86 system

In [ ]:
config_options = widgets.RadioButtons(
    options=['Pinned', 'Unpinned'],
    description='Create the environment from pinned or unpinned configs',
    disabled=False,
    layout=widgets.Layout(min_width='800px')
)
display(config_options)

In [ ]:
import yaml

pinned = 'Pinned' in config_options.value
if pinned:
    yaml_name = '.yaml'
else:
    yaml_name = 'hydrosar.yml'

conda_yaml_pth = list(env_config_dir.glob(yaml_name))[0]
with open(conda_yaml_pth, 'r') as f:
    env = yaml.safe_load(f)
    env_name = env['name']

env_prefix = conda_prefix / env_name

print(f'environment name: {env_name}')
print(f'environment prefix: {env_prefix}')

In [ ]:
!mamba clean -p -t --yes
!pip cache purge

In [ ]:
import shutil

# Create the environment
!mamba env create -f $conda_yaml_pth --prefix $env_prefix --force

# Provide access to the environment in notebooks as a Jupyter kernel named for the environment 
!{env_prefix}/bin/python -m ipykernel install --prefix {env_prefix} --name {env_name}

# Having created a well-named kernel, delete the default kernel named 'Python'
shutil.rmtree(env_prefix/'share/jupyter/kernels/python3')